# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />   

### Tarea 4

## Introducción

Considera un conjunto de imágenes de lugares etiquetadas en 15 categorías, el cual está dividido en
subconjuntos de entrenamiento (100 imágenes por categoría) y de prueba. Se extrajeron puntos
de interés por cada imagen usando el algoritmo SIFT [1] y sus vectores de características de 128
dimensiones. Calcula la representación de bolsa de características de las imágenes [2]. De forma
general, el procedimiento es el siguiente:

- 1. Se extraen puntos de interés de las imágenes y vectores de características que describan las
regiones de pixeles circundantes (estos vectores se proporcionan en el archivo del conjunto de
datos).
- 2. Se agrupan los vectores de características: comúnmente se emplea un algoritmo de agrupamiento como K-medias, donde cada centro es una palabra visual y al conjunto completo de centros se le conoce como vocabulario visual.
- 3. Se calcula la representación de bolsa de características de cada imagen como su histograma de frecuencias de palabras visuales.
- 4. Opcionalmente se aplican técnicas de reducción de dimensionalidad a los vectores y/o las bolsas de características.

## Objetivo

Entrena tres diferentes modelos no lineales de clasificación de imágenes (por ej. Gradient Boosted
Trees, Random Forests o Support Vector Machines) con las bolsas de características extraídas 3, construyendo un vocabulario visual de tamaño 10000:

In [1]:
import os
import joblib
import numpy as np
from tqdm import tqdm
from sklearn.cluster import MiniBatchKMeans
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.svm import SVC

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
BASE_DIR = "./data"

In [3]:
def delete_images(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".jpg"):
                os.remove(os.path.join(root, file))

In [4]:
delete_images(BASE_DIR)

# Crear dataset 

1. Se extraen puntos de interés de las imágenes y vectores de características que describan las
regiones de pixeles circundantes (estos vectores se proporcionan en el archivo del conjunto de
datos).

- Hay dos carpetas: train y test
- En train y test hay carpetas con la categoría de la imagen
- En cada carpeta hay una carpeta llamada features 
- En cada carpeta features hay un archivos .npy con las características de cada imagen

#### Queremos un dataframe train y un dataset test con las características de cada imagen y su categoría

| features (array)    | category | filename     |
|---------------------|----------|--------------|
| array(845, 128)     | playa    | img_001.npy  |
| array(612, 128)     | bosque   | img_003.npy  |
| array(790, 128)     | ciudad   | img_010.npy  |
| ...                 | ...      | ...          |

In [5]:


def create_dataset(base_dir, split):
    data = []
    split_path = os.path.join(base_dir, split)
    categories = sorted(os.listdir(split_path))
    
    for category in tqdm(categories, desc=f"Procesando categorías en {split}"):
        category_path = os.path.join(split_path, category)
        if not os.path.isdir(category_path):
            continue
        features_path = os.path.join(category_path, "features")
        if not os.path.exists(features_path):
            continue
        files = [fname for fname in os.listdir(features_path) if fname.endswith(".npy")]
        for fname in tqdm(files, desc=f"Procesando archivos en {category}", leave=False):
            fpath = os.path.join(features_path, fname)
            features = np.load(fpath)
            if features.shape[1] == 128: 
                data.append({"features": features, "category": category, "filename": fname.replace(".jpg.npy", "")})
    return pd.DataFrame(data)

# Crear datasets
train_df = create_dataset(BASE_DIR, "train")
test_df = create_dataset(BASE_DIR, "test")

Procesando categorías en test: 100%|██████████| 16/16 [00:00<00:00, 18.86it/s]


In [6]:
train_df

,features,category,filename
0,"[[0.00783572, 0.011618482, 0.0029721698, 0.0, ...",bedroom,image_0169
1,"[[0.03668815, 0.0009915717, 0.0, 0.0, 0.0, 0.0...",bedroom,image_0001
2,"[[0.0, 0.0047675804, 0.042908225, 0.0015891935...",bedroom,image_0179
3,"[[0.0047409413, 0.0027091093, 0.00067727733, 0...",bedroom,image_0146
4,"[[0.0045592706, 0.017857144, 0.0049392097, 0.0...",bedroom,image_0216
...,...,...,...
1495,"[[0.0006495615, 0.0016239039, 0.018187724, 0.0...",tallbuilding,image_0257
1496,"[[0.016569767, 0.013372093, 0.000872093, 0.000...",tallbuilding,image_0049
1497,"[[0.0007246377, 0.00036231885, 0.00036231885, ...",tallbuilding,image_0066
1498,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00...",tallbuilding,image_0076


## Vocabulario Visual

2. Se agrupan los vectores de características: comúnmente se emplea un algoritmo de agrupamiento como K-medias, donde cada centro es una palabra visual y al conjunto completo de centros se le conoce como vocabulario visual.

In [7]:
def create_vocabulary(df, n_clusters=1000):
    all_features = np.concatenate(df["features"].values)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=1000, random_state=42, verbose=2)
    kmeans.fit(all_features)
    return kmeans

In [8]:
model = create_vocabulary(train_df, n_clusters=10000)

Init 1/1 with method k-means++
Inertia for init 1/1: 207.25619506835938
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 1/103278: mean batch inertia: 0.006866801261901856
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 2/103278: mean batch inertia: 0.0070305862426757815, ewa inertia: 0.0070305862426757815
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 3/103278: mean batch inertia: 0.006997864246368408, ewa inertia: 0.007030522875909145
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 4/103278: mean batch inertia: 0.006895594120025635, ewa inertia: 0.007030261583813563
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 5/103278: mean batch inertia: 0.006941184043884278, ewa inertia: 0.007030089083467547
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 6/103278: mean batch inertia: 0.006793022155761719, ewa inertia: 0.0070296299988645075
[MiniBatchKMeans] Reassigning 500 cluster center

In [9]:
# Guardar el modelo

model_path = os.path.join("models","VOCAB10K", "kmeans_model.pkl")
joblib.dump(model, model_path)
print(f"Modelo guardado en: {model_path}")

Modelo guardado en: models/VOCAB10K/kmeans_model.pkl


## Calcular la representación de bolsa de características

3. Se calcula la representación de bolsa de características de cada imagen como su histograma de frecuencias de palabras visuales.

In [10]:
# Cargar el modelo
model = joblib.load(model_path)
print(f"Modelo cargado desde: {model_path}")

Modelo cargado desde: models/VOCAB10K/kmeans_model.pkl


In [11]:
def getBoVW(df, model):
    bow_representations = []
    for features in tqdm(df["features"], desc="Calculando BoVW"):
        clusters = model.predict(features)
        histogram, _ = np.histogram(clusters, bins=np.arange(model.n_clusters + 1))
        bow_representations.append(histogram)
    df["bow"] = bow_representations
    return df

# Calcular representación de bolsa de características para train y test
train_df = getBoVW(train_df, model)
test_df = getBoVW(test_df, model)

Calculando BoVW: 100%|██████████| 2985/2985 [00:37<00:00, 80.49it/s]


In [12]:
train_df[['bow', 'category']].to_csv(os.path.join(BASE_DIR, "train_bow.csv"), index=False)
test_df[['bow', 'category']].to_csv(os.path.join(BASE_DIR, "test_bow.csv"), index=False)

# Visualización

4. Opcionalmente se aplican técnicas de reducción de dimensionalidad a los vectores y/o las bolsas de características.

In [13]:
# PCA 

def viz(df):
    pca3 = PCA(n_components=3)
    pca2 = PCA(n_components=2)
    r3 = pca3.fit_transform(np.array(df['bow'].tolist()))
    r2 = pca2.fit_transform(np.array(df['bow'].tolist()))

    fig = px.scatter_3d(
        x=r3[:, 0],
        y=r3[:, 1],
        z=r3[:, 2],
        color=df['category'],
        title="Visualización de PCA en 3D",
        labels={'x': 'Componente 1', 'y': 'Componente 2', 'z': 'Componente 3'},
    )
    fig.update_traces(marker=dict(size=5))
    fig.update_layout(scene=dict(
        xaxis_title='Componente 1',
        yaxis_title='Componente 2',
        zaxis_title='Componente 3'
    ))
    fig.show()
    fig = px.scatter(
        x=r2[:, 0],
        y=r2[:, 1],
        color=df['category'],
        title="Visualización de PCA en 2D",
        labels={'x': 'Componente 1', 'y': 'Componente 2'},
    )
    fig.update_traces(marker=dict(size=5))
    fig.update_layout(
        xaxis_title='Componente 1',
        yaxis_title='Componente 2'
    )
    fig.show()
    return r3, r2

In [14]:
r3_train, r2_train = viz(train_df)


In [15]:
r3_test, r2_test = viz(test_df)

# Entrenamiento 1

Usando el vocabulario visual de tamaño 10000

In [16]:
def eval_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Accuracy Score:", accuracy_score(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    fig = px.imshow(confusion_matrix(y_test, y_pred),
                    labels=dict(x="Predicted", y="True"),
                    x=sorted(test_df['category'].unique()),
                    y=sorted(test_df['category'].unique()),
                    title="Confusion Matrix",
                    color_continuous_scale='Viridis')
    fig.update_xaxes(side="top")
    fig.show()
    return y_pred

In [17]:
X_train = np.array(train_df['bow'].tolist())
y_train = train_df['category'].values
X_test = np.array(test_df['bow'].tolist())
y_test = test_df['category'].values

### SVC

In [18]:
svc_model = SVC(kernel='rbf', C=1, gamma='scale')
svc_model.fit(X_train, y_train)
joblib.dump(svc_model, os.path.join("models", "VOCAB10K", "svc_model.pkl"))
svc_model = joblib.load(os.path.join("models", "VOCAB10K", "svc_model.pkl"))
y_pred = eval_model(svc_model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

     bedroom       0.20      0.10      0.14       116
       coast       0.60      0.70      0.65       260
      forest       0.71      0.81      0.76       228
     highway       0.51      0.59      0.55       160
  industrial       0.23      0.25      0.24       211
  insidecity       0.58      0.40      0.48       208
     kitchen       0.20      0.78      0.32       110
  livingroom       0.49      0.42      0.45       189
    mountain       0.62      0.35      0.45       274
      office       0.76      0.30      0.43       115
 opencountry       0.47      0.41      0.44       310
       store       0.55      0.47      0.51       215
      street       0.53      0.40      0.45       192
      suburb       0.47      0.85      0.60       141
tallbuilding       0.79      0.50      0.61       256

    accuracy                           0.49      2985
   macro avg       0.51      0.49      0.47      2985
wei

### Random Forest



In [19]:
rf_model = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [1000]}
rf_grid = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)
print("Mejor número de árboles:", rf_grid.best_params_['n_estimators'])
joblib.dump(rf_grid, os.path.join("models", "VOCAB10K", "rf_model.pkl"))
rf_grid = joblib.load(os.path.join("models", "VOCAB10K", "rf_model.pkl"))
y_pred = eval_model(rf_grid, X_test, y_test)

Mejor número de árboles: 1000
Classification Report:
              precision    recall  f1-score   support

     bedroom       0.39      0.10      0.16       116
       coast       0.39      0.85      0.54       260
      forest       0.42      0.97      0.59       228
     highway       0.36      0.59      0.45       160
  industrial       0.38      0.11      0.17       211
  insidecity       0.45      0.32      0.37       208
     kitchen       0.17      0.51      0.26       110
  livingroom       0.61      0.11      0.18       189
    mountain       0.51      0.28      0.36       274
      office       0.27      0.80      0.41       115
 opencountry       0.42      0.18      0.26       310
       store       0.61      0.13      0.21       215
      street       0.57      0.14      0.22       192
      suburb       0.79      0.48      0.60       141
tallbuilding       0.67      0.62      0.64       256

    accuracy                           0.41      2985
   macro avg       0.47    

### Gradient Boosted Trees

In [20]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

xgbc_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    njobs=-1,  # Usar todos los núcleos disponibles
    use_label_encoder=False,  # Evitar advertencia de codificación
)

xgbc_model.fit(X_train, y_train_encoded)
joblib.dump(xgbc_model, os.path.join("models", "VOCAB10K", "xgbc_model.pkl"))
xgbc_model = joblib.load(os.path.join("models", "VOCAB10K", "xgbc_model.pkl"))
y_pred_xgbc = eval_model(xgbc_model, X_test, y_test_encoded)
y_pred_decoded = label_encoder.inverse_transform(y_pred_xgbc)


/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning:

[20:38:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "njobs", "use_label_encoder" } are not used.




Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.24      0.21       116
           1       0.48      0.59      0.53       260
           2       0.57      0.63      0.60       228
           3       0.45      0.51      0.47       160
           4       0.22      0.16      0.18       211
           5       0.36      0.40      0.38       208
           6       0.22      0.44      0.30       110
           7       0.38      0.24      0.30       189
           8       0.36      0.31      0.33       274
           9       0.36      0.51      0.42       115
          10       0.38      0.29      0.33       310
          11       0.34      0.27      0.30       215
          12       0.31      0.23      0.27       192
          13       0.42      0.50      0.45       141
          14       0.60      0.57      0.58       256

    accuracy                           0.39      2985
   macro avg       0.38      0.39      0.38      2985
wei

# Entrenamiento 2

Utilizando PCA para reducir la dimensionalidad de las características a 1000 dimensiones.

In [21]:
# Reducir dimensionalidad con PCA
pca = PCA(n_components=1000) 

X_train_pca = pca.fit_transform(np.array(train_df['bow'].tolist()))
X_test_pca = pca.transform(np.array(test_df['bow'].tolist()))

y_train = train_df['category'].values
y_test = test_df['category'].values

In [22]:
svc_model = SVC(kernel='rbf', C=1, gamma='scale')
svc_model.fit(X_train_pca, y_train)
joblib.dump(svc_model, os.path.join("models", "VOCAB10K", "svc_modelPCA.pkl"))
svc_model = joblib.load(os.path.join("models", "VOCAB10K", "svc_modelPCA.pkl"))
y_pred = eval_model(svc_model, X_test_pca, y_test)

Classification Report:
              precision    recall  f1-score   support

     bedroom       0.50      0.01      0.02       116
       coast       0.40      0.86      0.54       260
      forest       0.83      0.64      0.72       228
     highway       0.25      0.68      0.37       160
  industrial       0.11      0.04      0.06       211
  insidecity       0.25      0.04      0.07       208
     kitchen       0.10      0.97      0.19       110
  livingroom       1.00      0.04      0.07       189
    mountain       0.56      0.03      0.06       274
      office       0.39      0.10      0.15       115
 opencountry       0.27      0.43      0.33       310
       store       0.00      0.00      0.00       215
      street       0.31      0.04      0.07       192
      suburb       0.00      0.00      0.00       141
tallbuilding       0.88      0.27      0.41       256

    accuracy                           0.28      2985
   macro avg       0.39      0.28      0.20      2985
wei

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
rf_model = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [100, 500, 1000]}
rf_grid = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
rf_grid.fit(X_train_pca, y_train)
print("Mejor número de árboles:", rf_grid.best_params_['n_estimators'])
joblib.dump(rf_grid, os.path.join("models", "VOCAB10K", "rf_modelPCA.pkl"))
rf_grid = joblib.load(os.path.join("models", "VOCAB10K", "rf_modelPCA.pkl"))
y_pred = eval_model(rf_grid, X_test_pca, y_test)

Classification Report:
              precision    recall  f1-score   support

     bedroom       0.32      0.15      0.20       116
       coast       0.35      0.83      0.50       260
      forest       0.65      0.81      0.72       228
     highway       0.34      0.72      0.47       160
  industrial       0.33      0.08      0.13       211
  insidecity       0.44      0.53      0.48       208
     kitchen       0.19      0.71      0.30       110
  livingroom       0.60      0.15      0.24       189
    mountain       0.58      0.44      0.50       274
      office       0.41      0.63      0.49       115
 opencountry       0.51      0.06      0.11       310
       store       0.76      0.14      0.24       215
      street       0.58      0.26      0.35       192
      suburb       0.78      0.67      0.72       141
tallbuilding       0.58      0.64      0.61       256

    accuracy                           0.44      2985
   macro avg       0.50      0.45      0.41      2985
wei

In [25]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

xgbc_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    njobs=-1,  # Usar todos los núcleos disponibles
    use_label_encoder=False,  # Evitar advertencia de codificación
)

xgbc_model.fit(X_train_pca, y_train_encoded)
joblib.dump(xgbc_model, os.path.join("models", "VOCAB10K", "xgbc_model.pkl"))
xgbc_model = joblib.load(os.path.join("models", "VOCAB10K", "xgbc_model.pkl"))
y_pred_xgbc = eval_model(xgbc_model, X_test_pca, y_test_encoded)
y_pred_decoded = label_encoder.inverse_transform(y_pred_xgbc)


/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning:

[20:45:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "njobs", "use_label_encoder" } are not used.




Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.27      0.26       116
           1       0.58      0.71      0.64       260
           2       0.64      0.93      0.76       228
           3       0.46      0.65      0.54       160
           4       0.34      0.14      0.20       211
           5       0.45      0.49      0.47       208
           6       0.26      0.45      0.33       110
           7       0.50      0.45      0.47       189
           8       0.56      0.43      0.49       274
           9       0.50      0.58      0.54       115
          10       0.52      0.35      0.42       310
          11       0.51      0.45      0.48       215
          12       0.60      0.45      0.52       192
          13       0.66      0.82      0.73       141
          14       0.65      0.62      0.63       256

    accuracy                           0.52      2985
   macro avg       0.50      0.52      0.50      2985
wei

# Entrenamiento 3

Utilizando un vocabulario visual de tamaño 5000

In [27]:
# Obtener un vocabulario de 5000 palabras

model5K = create_vocabulary(train_df, n_clusters=5000)
model_path = os.path.join("models","VOCAB5K", "kmeans_model.pkl")
joblib.dump(model, model_path)
print(f"Modelo guardado en: {model_path}")
model5K = joblib.load(model_path)
print(f"Modelo cargado desde: {model_path}")

Init 1/1 with method k-means++
Inertia for init 1/1: 112.186279296875
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 1/103278: mean batch inertia: 0.0074065465927124026
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 2/103278: mean batch inertia: 0.007504017353057862, ewa inertia: 0.007504017353057862
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 3/103278: mean batch inertia: 0.00738625431060791, ewa inertia: 0.007503789302692005
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 4/103278: mean batch inertia: 0.007339343070983887, ewa inertia: 0.0075034708494444964
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 5/103278: mean batch inertia: 0.007266904830932617, ewa inertia: 0.007503012734861614
[MiniBatchKMeans] Reassigning 500 cluster centers.
Minibatch step 6/103278: mean batch inertia: 0.00698483419418335, ewa inertia: 0.007502009272286919
[MiniBatchKMeans] Reassigning 500 cluster centers.
Mi

In [28]:
train_df5k = getBoVW(train_df, model5K)
test_df5k = getBoVW(test_df, model5K)

Calculando BoVW: 100%|██████████| 2985/2985 [00:38<00:00, 78.48it/s]


In [29]:
X_train = np.array(train_df5k['bow'].tolist())
y_train = train_df5k['category'].values
X_test = np.array(test_df5k['bow'].tolist())
y_test = test_df5k['category'].values

In [30]:
svc_model = SVC(kernel='rbf', C=1, gamma='scale')
svc_model.fit(X_train, y_train)
joblib.dump(svc_model, os.path.join("models", "VOCAB5K", "svc_model.pkl"))
svc_model = joblib.load(os.path.join("models", "VOCAB5K", "svc_model.pkl"))
y_pred = eval_model(svc_model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

     bedroom       0.20      0.10      0.14       116
       coast       0.60      0.70      0.65       260
      forest       0.71      0.81      0.76       228
     highway       0.51      0.59      0.55       160
  industrial       0.23      0.25      0.24       211
  insidecity       0.58      0.40      0.48       208
     kitchen       0.20      0.78      0.32       110
  livingroom       0.49      0.42      0.45       189
    mountain       0.62      0.35      0.45       274
      office       0.76      0.30      0.43       115
 opencountry       0.47      0.41      0.44       310
       store       0.55      0.47      0.51       215
      street       0.53      0.40      0.45       192
      suburb       0.47      0.85      0.60       141
tallbuilding       0.79      0.50      0.61       256

    accuracy                           0.49      2985
   macro avg       0.51      0.49      0.47      2985
wei

In [32]:
rf_model = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [1000]}
rf_grid = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)
print("Mejor número de árboles:", rf_grid.best_params_['n_estimators'])
joblib.dump(rf_grid, os.path.join("models", "VOCAB5K", "rf_model.pkl"))
rf_grid = joblib.load(os.path.join("models", "VOCAB5K", "rf_model.pkl"))
y_pred = eval_model(rf_grid, X_test, y_test)

Mejor número de árboles: 1000
Classification Report:
              precision    recall  f1-score   support

     bedroom       0.39      0.10      0.16       116
       coast       0.39      0.85      0.54       260
      forest       0.42      0.97      0.59       228
     highway       0.36      0.59      0.45       160
  industrial       0.38      0.11      0.17       211
  insidecity       0.45      0.32      0.37       208
     kitchen       0.17      0.51      0.26       110
  livingroom       0.61      0.11      0.18       189
    mountain       0.51      0.28      0.36       274
      office       0.27      0.80      0.41       115
 opencountry       0.42      0.18      0.26       310
       store       0.61      0.13      0.21       215
      street       0.57      0.14      0.22       192
      suburb       0.79      0.48      0.60       141
tallbuilding       0.67      0.62      0.64       256

    accuracy                           0.41      2985
   macro avg       0.47    

In [33]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

xgbc_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    njobs=-1,  # Usar todos los núcleos disponibles
    use_label_encoder=False,  # Evitar advertencia de codificación
)

xgbc_model.fit(X_train, y_train_encoded)
joblib.dump(xgbc_model, os.path.join("models", "VOCAB5K", "xgbc_model.pkl"))
xgbc_model = joblib.load(os.path.join("models", "VOCAB5K", "xgbc_model.pkl"))
y_pred_xgbc = eval_model(xgbc_model, X_test, y_test_encoded)
y_pred_decoded = label_encoder.inverse_transform(y_pred_xgbc)


/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning:

[20:53:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "njobs", "use_label_encoder" } are not used.




Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.24      0.21       116
           1       0.48      0.59      0.53       260
           2       0.57      0.63      0.60       228
           3       0.45      0.51      0.47       160
           4       0.22      0.16      0.18       211
           5       0.36      0.40      0.38       208
           6       0.22      0.44      0.30       110
           7       0.38      0.24      0.30       189
           8       0.36      0.31      0.33       274
           9       0.36      0.51      0.42       115
          10       0.38      0.29      0.33       310
          11       0.34      0.27      0.30       215
          12       0.31      0.23      0.27       192
          13       0.42      0.50      0.45       141
          14       0.60      0.57      0.58       256

    accuracy                           0.39      2985
   macro avg       0.38      0.39      0.38      2985
wei

## Entrenamiento 4

Utilizando un vocabulario visual de tamaño 5K y PCA para reducir la dimensionalidad de las características a 100 dimensiones.

In [34]:
pca = PCA(n_components=100) 

X_train_pca = pca.fit_transform(np.array(train_df5k['bow'].tolist()))
X_test_pca = pca.transform(np.array(test_df5k['bow'].tolist()))

y_train = train_df5k['category'].values
y_test = test_df5k['category'].values

In [35]:
svc_model = SVC(kernel='rbf', C=1, gamma='scale')
svc_model.fit(X_train_pca, y_train)
joblib.dump(svc_model, os.path.join("models", "VOCAB5K", "svc_modelPCA.pkl"))
svc_model = joblib.load(os.path.join("models", "VOCAB5K", "svc_modelPCA.pkl"))
y_pred = eval_model(svc_model, X_test_pca, y_test)

Classification Report:
              precision    recall  f1-score   support

     bedroom       0.30      0.19      0.23       116
       coast       0.68      0.66      0.67       260
      forest       0.73      0.86      0.79       228
     highway       0.52      0.73      0.61       160
  industrial       0.30      0.41      0.35       211
  insidecity       0.61      0.49      0.55       208
     kitchen       0.29      0.77      0.42       110
  livingroom       0.59      0.35      0.44       189
    mountain       0.67      0.60      0.64       274
      office       0.55      0.62      0.58       115
 opencountry       0.57      0.54      0.56       310
       store       0.67      0.47      0.56       215
      street       0.47      0.59      0.53       192
      suburb       0.87      0.78      0.82       141
tallbuilding       0.84      0.40      0.54       256

    accuracy                           0.56      2985
   macro avg       0.58      0.56      0.55      2985
wei

In [38]:
rf_model = RandomForestClassifier(random_state=42)
param_grid = {'n_estimators': [100, 500, 1000]}
rf_grid = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')
rf_grid.fit(X_train_pca, y_train)
print("Mejor número de árboles:", rf_grid.best_params_['n_estimators'])
joblib.dump(rf_grid, os.path.join("models", "VOCAB5K", "rf_modelPCA.pkl"))
rf_grid = joblib.load(os.path.join("models", "VOCAB5K", "rf_modelPCA.pkl"))
y_pred = eval_model(rf_grid, X_test_pca, y_test)

Mejor número de árboles: 500
Classification Report:
              precision    recall  f1-score   support

     bedroom       0.31      0.23      0.27       116
       coast       0.62      0.66      0.64       260
      forest       0.66      0.84      0.74       228
     highway       0.47      0.72      0.57       160
  industrial       0.43      0.16      0.23       211
  insidecity       0.53      0.59      0.56       208
     kitchen       0.28      0.53      0.37       110
  livingroom       0.61      0.43      0.50       189
    mountain       0.62      0.49      0.55       274
      office       0.51      0.70      0.59       115
 opencountry       0.52      0.39      0.45       310
       store       0.58      0.46      0.51       215
      street       0.44      0.71      0.54       192
      suburb       0.68      0.84      0.75       141
tallbuilding       0.77      0.55      0.64       256

    accuracy                           0.55      2985
   macro avg       0.54     

In [39]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

xgbc_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    njobs=-1,  # Usar todos los núcleos disponibles
    use_label_encoder=False,  # Evitar advertencia de codificación
)

xgbc_model.fit(X_train_pca, y_train_encoded)
joblib.dump(xgbc_model, os.path.join("models", "VOCAB5K", "xgbc_model.pkl"))
xgbc_model = joblib.load(os.path.join("models", "VOCAB5K", "xgbc_model.pkl"))
y_pred_xgbc = eval_model(xgbc_model, X_test_pca, y_test_encoded)
y_pred_decoded = label_encoder.inverse_transform(y_pred_xgbc)

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning:

[20:59:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "njobs", "use_label_encoder" } are not used.




Classification Report:
              precision    recall  f1-score   support

           0       0.38      0.33      0.35       116
           1       0.63      0.62      0.63       260
           2       0.66      0.92      0.77       228
           3       0.49      0.66      0.56       160
           4       0.30      0.27      0.28       211
           5       0.53      0.50      0.51       208
           6       0.30      0.48      0.37       110
           7       0.53      0.46      0.49       189
           8       0.65      0.47      0.55       274
           9       0.49      0.66      0.56       115
          10       0.52      0.45      0.48       310
          11       0.57      0.41      0.48       215
          12       0.50      0.60      0.54       192
          13       0.77      0.86      0.81       141
          14       0.74      0.58      0.65       256

    accuracy                           0.55      2985
   macro avg       0.54      0.55      0.54      2985
wei

# Discusión

#### 1. ¿Cuáles consideras que son las ventajas y desventajas de los modelos de clasificación generados y por qué?

En general, los modelos de clasificación generados son buenos para clasificar imágenes en diferentes categorías. Sin embargo, cada modelo tiene sus propias ventajas y desventajas.
- **Gradient Boosted Trees**: Este modelo es muy efectivo para datos tabulares y puede manejar características no lineales. Sin embargo, puede ser propenso al sobreajuste si no se ajustan adecuadamente los hiperparámetros.
- **Random Forests**: Este modelo es robusto y menos propenso al sobreajuste que los árboles de decisión individuales. Sin embargo, puede ser menos efectivo en datos con alta dimensionalidad.
- **SVC**: Este modelo es muy efectivo para datos de alta dimensionalidad y puede manejar características no lineales. Sin embargo, puede ser lento para conjuntos de datos grandes y requiere una buena selección de hiperparámetros.




#### 2. En el contexto de procesamiento del lenguaje natural, define el concepto de stop word y sugiere cómo lo llevarías a cabo en este problema de clasificación de imágenes.

(Por fin puedo sacar a relucir que también estoy inscrito en el curso de procesamiento de lenguaje natural) 

En el contexto de procesamiento del lenguaje natural, una stop word es una palabra que se considera irrelevante para el análisis y se elimina del texto. Estas palabras suelen ser artículos, preposiciones y pronombres que no aportan mucho significado al contenido. 

En el problema de clasificación de imágenes, podríamos considerar eliminar características que no aporten información relevante para la clasificación, como características que aparecen con poca frecuencia o que son redundantes. Esto podría ayudar a mejorar la precisión del modelo y reducir el tiempo de entrenamiento.

Para llevar a cabo esto, podríamos aplicar técnicas de selección de características, como la eliminación de características con baja varianza o la selección de características basada en la importancia de las características. 

Supongo que la reducción de dimensionalidad podría incluso llegar a considerarse un tipo muy especifico de eliminación de stop-words.